# Milestone P4 : Creative Extension

## Libraries

In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import math
import matplotlib.image as mpimg
%matplotlib inline

## The data

In [2]:
DATA_FOLDER = 'data/'

GOWALLA_EDGES_DATASET = DATA_FOLDER+"loc-gowalla_edges.txt.gz"
GOWALLA_CHECKINS_DATASET = DATA_FOLDER+"loc-gowalla_totalCheckins.txt.gz"
BRIGHTKITE_EDGES_DATASET = DATA_FOLDER+"loc-brightkite_edges.txt.gz"
BRIGHTKITE_CHECKINS_DATASET = DATA_FOLDER+"loc-brightkite_totalCheckins.txt.gz"
POPULATION_DATASET = DATA_FOLDER+"pop_per_country.csv"
BRIGHTKITE_HOME_LOCATIONS = DATA_FOLDER+"new_home_loc_B_country.csv"
GOWALLA_HOME_LOCATIONS = DATA_FOLDER+"new_home_loc_G_country.csv"
COUNTRY_CODES = DATA_FOLDER+"countrycodes.csv"

gowalla_edges = pd.read_csv(GOWALLA_EDGES_DATASET, compression = 'gzip', delimiter = '\t', names = ['userA', 'userB'], header=None)
gowalla_checkins = pd.read_csv(GOWALLA_CHECKINS_DATASET, compression = 'gzip', delimiter = '\t', names = ['user', 'checkin_time', 'latitude', 'longitude', 'location_id'], header=None)
brightkite_edges = pd.read_csv(BRIGHTKITE_EDGES_DATASET, compression = 'gzip', delimiter = '\t', names = ['userA', 'userB'], header=None)
brightkite_checkins = pd.read_csv(BRIGHTKITE_CHECKINS_DATASET, compression = 'gzip', delimiter = '\t', names = ['user', 'checkin_time', 'latitude', 'longitude', 'location_id'], header=None)
population = pd.read_csv(POPULATION_DATASET, index_col = 0, names = ['country', 'population'], header = 0)
brightkite_home_locations = pd.read_csv(BRIGHTKITE_HOME_LOCATIONS, sep =',')
gowalla_home_locations = pd.read_csv(GOWALLA_HOME_LOCATIONS, sep =',')
country_codes = pd.read_csv(COUNTRY_CODES, sep =',')

In [3]:
population.head(3)

,country,population
0,afghanistan,31056997
1,albania,3581655
2,algeria,32930091


In [4]:
population.sort_values(by = ['population'], ascending = False).head(10)

,country,population
24,china,1313973713
51,india,1095351995
127,united states,298444215
52,indonesia,245452739
15,brazil,188078227
93,pakistan,165803560
9,bangladesh,147365352
101,russia,142893540
91,nigeria,131859731
59,japan,127463611


In [5]:
brightkite_home_locations.head(3)

,user,lat,lon,country
0,0,39.747913,-104.968944,US
1,1,37.599899,-122.372723,US
2,2,39.738874,-104.954143,US


In [6]:
gowalla_home_locations.head(3)

,user,lat,lon,country
0,0,30.259167,-97.750223,US
1,1,50.381005,3.430710,FR
2,2,34.052243,-118.327641,US


In [7]:
country_codes.head()

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [8]:
gowalla_home_locations = pd.merge(gowalla_home_locations, country_codes, left_on = 'country', right_on = 'Code', how = 'left')
gowalla_home_locations.drop(columns = ['country', 'Code'], inplace = True)
gowalla_home_locations.rename(columns = {'Name':'country'}, inplace = True)

brightkite_home_locations = pd.merge(brightkite_home_locations, country_codes, left_on = 'country', right_on = 'Code', how = 'left')
brightkite_home_locations.drop(columns = ['country', 'Code'], inplace = True)
brightkite_home_locations.rename(columns = {'Name':'country'}, inplace = True)

In [9]:
brightkite_home_locations.head()

,user,lat,lon,country
0,0,39.747913,-104.968944,United States
1,1,37.599899,-122.372723,United States
2,2,39.738874,-104.954143,United States
3,3,37.780875,-122.406171,United States
4,4,60.174186,24.937477,Finland


## Step 1 : defining the geographic boundaries of our study

Before starting the study, we have to determine with which countries it would relevant to work. To estimate the quantity of data available in each country, we compute the number of homes, as defined in the replication, (i.e. users) that are located in each country. 

The point is to work only on countries in which the amount of data seems to be enough to have robust conclusions. 

In [10]:
brightkite_home_locations['country'] = brightkite_home_locations['country'].str.lower()
brightkite_home_locations['country'] = brightkite_home_locations['country'].str.lstrip()

gowalla_home_locations['country'] = gowalla_home_locations['country'].str.lower()
gowalla_home_locations['country'] = gowalla_home_locations['country'].str.lstrip()

In [11]:
brightkite_homes_per_country = brightkite_home_locations.groupby(['country']).count().sort_values(by = ['lat'], ascending = False)
brightkite_homes_per_country.drop(columns = ['lon'], inplace = True)
brightkite_homes_per_country.rename(columns={'lat':'count'}, inplace = True)
brightkite_homes_per_country.reset_index(inplace = True)

gowalla_homes_per_country = gowalla_home_locations.groupby(['country']).count().sort_values(by = ['lat'], ascending = False)
gowalla_homes_per_country.drop(columns = ['lon'], inplace = True)
gowalla_homes_per_country.rename(columns={'lat':'count'}, inplace = True)
gowalla_homes_per_country.reset_index(inplace = True)

gowalla_homes_per_country.head(10)

,country,user,count
0,united states,50639,50639
1,sweden,18029,18029
2,united kingdom,5435,5435
3,germany,5213,5213
4,norway,3662,3662
5,canada,2210,2210
6,saudi arabia,1917,1917
7,thailand,1660,1660
8,belgium,1569,1569
9,australia,1311,1311


In [12]:
brightkite_homes_per_country.head(10)

,country,user,count
0,united states,30669,30669
1,united kingdom,3243,3243
2,japan,2860,2860
3,australia,1452,1452
4,canada,1409,1409
5,germany,1344,1344
6,sweden,833,833
7,netherlands,789,789
8,italy,614,614
9,norway,593,593


In [13]:
brightkite_homes_per_country = brightkite_homes_per_country.merge(population, how = 'inner', on = ['country'])
gowalla_homes_per_country = gowalla_homes_per_country.merge(population, how = 'inner', on = ['country'])

In [14]:
brightkite_homes_per_country.head(3)

,country,user,count,population
0,united states,30669,30669,298444215
1,united kingdom,3243,3243,60609153
2,japan,2860,2860,127463611


In [15]:
gowalla_homes_per_country.head(3)

,country,user,count,population
0,united states,50639,50639,298444215
1,sweden,18029,18029,9016596
2,united kingdom,5435,5435,60609153


The number of home per country is normalized by the population of the country. 

In [16]:
brightkite_homes_per_country['count_normalized'] = brightkite_homes_per_country['count']/brightkite_homes_per_country['population']
gowalla_homes_per_country['count_normalized'] = gowalla_homes_per_country['count']/gowalla_homes_per_country['population']

In [17]:
brightkite_homes_per_country.sort_values(by = 'count_normalized', ascending = False).head(10)

,country,user,count,population,count_normalized
9,norway,593,593,4610820,0.000129
0,united states,30669,30669,298444215,0.000103
6,sweden,833,833,9016596,0.000092
40,luxembourg,40,40,474413,0.000084
3,australia,1452,1452,20264082,0.000072
12,finland,367,367,5231372,0.000070
1,united kingdom,3243,3243,60609153,0.000054
7,netherlands,789,789,16491461,0.000048
4,canada,1409,1409,33098932,0.000043
36,estonia,53,53,1324333,0.000040


In [18]:
gowalla_homes_per_country.sort_values(by = 'count_normalized', ascending = False).head(10)

,country,user,count,population,count_normalized
1,sweden,18029,18029,9016596,0.002000
4,norway,3662,3662,4610820,0.000794
29,luxembourg,172,172,474413,0.000363
0,united states,50639,50639,298444215,0.000170
8,belgium,1569,1569,10379067,0.000151
11,switzerland,994,994,7523934,0.000132
17,denmark,607,607,5450661,0.000111
2,united kingdom,5435,5435,60609153,0.000090
21,singapore,392,392,4492150,0.000087
10,netherlands,1259,1259,16491461,0.000076


Let's say that we will only work with countries which count_normalized value is higher to a certain rate. 

In [19]:
rate = 0.00001
brightkite_countries = brightkite_homes_per_country[brightkite_homes_per_country.count_normalized >= rate]
gowalla_countries = gowalla_homes_per_country[gowalla_homes_per_country.count_normalized >= rate]
print('With this rate,', len(brightkite_countries), 'countries are taken in the study for the Brightkite dataset.')
print('With this rate,', len(gowalla_countries), 'countries are taken in the study for the Gowalla dataset.')

With this rate, 28 countries are taken in the study for the Brightkite dataset.
With this rate, 38 countries are taken in the study for the Gowalla dataset.


In [20]:
countries = pd.merge(brightkite_countries, gowalla_countries, how = 'inner', on = ['country'], suffixes=('_brightkite', '_gowalla')) # countries that are ok both for the Gowalla and Brightkite datasets
print('With this rate,', len(countries), 'countries are taken in the study for both datasets.')

With this rate, 25 countries are taken in the study for both datasets.


In [21]:
countries.head(5)

,country,user_brightkite,count_brightkite,population_brightkite,count_normalized_brightkite,user_gowalla,count_gowalla,population_gowalla,count_normalized_gowalla
0,united states,30669,30669,298444215,0.000103,50639,50639,298444215,0.000170
1,united kingdom,3243,3243,60609153,0.000054,5435,5435,60609153,0.000090
2,australia,1452,1452,20264082,0.000072,1311,1311,20264082,0.000065
3,canada,1409,1409,33098932,0.000043,2210,2210,33098932,0.000067
4,germany,1344,1344,82422299,0.000016,5213,5213,82422299,0.000063


## Distance from home

In [22]:
gowalla_checkins = pd.merge(gowalla_checkins, gowalla_home_locations, how = 'left', on = ['user'])
brightkite_checkins = pd.merge(brightkite_checkins, brightkite_home_locations, how = 'left', on = ['user'])

In [23]:
gowalla_checkins.rename(columns={'lat':'home_lat','lon':'home_lon'}, inplace = True)
brightkite_checkins.rename(columns={'lat':'home_lat','lon':'home_lon'}, inplace = True)

In [24]:
import haversine as hvrs

In [25]:
def distance_from_home(row):
    '''computes the distance between the check-in and the user's home'''
    lat1 = row['latitude']
    lat2 = row['home_lat']
    long1 = row['longitude']
    long2 = row['home_lon']
    return(hvrs.haversine((lat1, long1), (lat2, long2)))

In [26]:
gowalla_checkins['distance_from_home'] = gowalla_checkins.apply(lambda row: distance_from_home(row), axis = 1)

In [27]:
brightkite_checkins['distance_from_home'] = brightkite_checkins.apply(lambda row: distance_from_home(row), axis = 1)

In [28]:
brightkite_checkins.head(1)

,user,checkin_time,latitude,longitude,location_id,home_lat,home_lon,country,distance_from_home
0,0,2010-10-17T01:48:53Z,39.747652,-104.99251,88c46bf20db295831bd2d1718ad7e6f5,39.747913,-104.968944,united states,2.014929


In [29]:
distance = 200 #km
gowalla_far_checkins = gowalla_checkins[gowalla_checkins.distance_from_home >= distance]
brightkite_far_checkins = brightkite_checkins[brightkite_checkins.distance_from_home >= distance]
len(gowalla_far_checkins), len(brightkite_far_checkins)

(942731, 591617)

In [30]:
gowalla_far_checkins_per_country = gowalla_far_checkins.groupby(['country']).count().sort_values(by = 'user', ascending = False)
gowalla_far_checkins_per_country.rename(columns = {'user':'count'}, inplace = True)
gowalla_far_checkins_per_country.drop(gowalla_far_checkins_per_country.columns.difference(['count']), 1, inplace=True)
gowalla_far_checkins_per_country.head(10)

,count
country,
united states,634969
sweden,103647
germany,39038
united kingdom,29178
norway,23863
canada,15476
saudi arabia,12494
australia,7230
france,6171


In [31]:
brightkite_far_checkins_per_country = brightkite_far_checkins.groupby(['country']).count().sort_values(by = 'user', ascending = False)
brightkite_far_checkins_per_country.rename(columns = {'user':'count'}, inplace = True)
brightkite_far_checkins_per_country.drop(brightkite_far_checkins_per_country.columns.difference(['count']), 1, inplace=True)
brightkite_far_checkins_per_country.head(10)

,count
country,
united states,399176
japan,65018
united kingdom,17390
germany,11122
australia,10638
canada,9383
sweden,8656
france,7506
spain,7043
